# 0. Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

This line installs the required packages for the tutorial, specifically mediapipe (which is a library for hand, pose, and face detection) and opencv-python (a library for computer vision tasks).

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Here, the necessary libraries are imported.
cv2: This is the OpenCV library which is used for various computer vision tasks.
mediapipe as mp: Mediapipe is imported with the alias mp.
numpy as np: Numpy is imported with the alias np.
mp_drawing: This is a utility from Mediapipe to help with drawing on images.
mp_pose: This helps in detecting human poses using Mediapipe.

In [ ]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed 1', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

This block of code sets up a video feed using OpenCV.
cv2.VideoCapture(0): This initializes the video capture with the primary webcam of the computer.
while cap.isOpened(): Continuously capture frames as long as the webcam is active.
ret, frame = cap.read(): Read a frame from the webcam. ret is a boolean that indicates if the frame was successfully grabbed.
cv2.imshow('Mediapipe Feed', frame): This displays the captured frame in a window named 'Mediapipe Feed'.
if cv2.waitKey(10) & 0xFF == ord('q'): This checks if the 'q' key is pressed. If pressed, it breaks out of the loop to stop capturing.
cap.release(): Releases the video capture object.
cv2.destroyAllWindows(): Closes all OpenCV windows.

# 1. Make Detections

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.1, min_tracking_confidence=0.1) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

This block of code captures video from the webcam and processes each frame using Mediapipe to detect human poses.
The code inside the with statement initializes the pose detection from Mediapipe.
Each frame from the webcam is converted from BGR (Blue, Green, Red - the default format of OpenCV) to RGB (Red, Green, Blue - the format Mediapipe uses).
pose.process(image): Detects the pose in the image.
The detected landmarks (keypoints of the pose) are then drawn on the image using mp_drawing.draw_landmarks(...).
The processed image is then displayed using cv2.imshow(...).
The loop continues until the 'q' key is pressed.

In [ ]:
mp_drawing.DrawingSpec??

mp_drawing.DrawingSpec??
This line is likely an interactive command to inspect the documentation or source code of the DrawingSpec class from Mediapipe. In Jupyter notebooks, appending ?? to a function or class name displays its source code and docstring.

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.1, min_tracking_confidence=0.1) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

This block of code is very similar to the one we discussed earlier. The main addition is the extraction of landmarks.
The line landmarks = results.pose_landmarks.landmark extracts the pose landmarks (keypoints) detected by Mediapipe. These landmarks represent different parts of the body, like the nose, eyes, shoulders, elbows, etc.
The extracted landmarks are then printed to the console.
The try and except block is used to handle any potential errors that might arise when extracting landmarks. If there's an error (e.g., no landmarks detected), the code inside the except block is executed, which in this case, does nothing (using pass).

1. Pose Landmarks in Mediapipe:
Mediapipe's Pose solution provides 33 landmarks across the body, from the nose tip to the feet. Each landmark has a unique ID and corresponds to a specific part of the body. For instance:

mp_pose.PoseLandmark.LEFT_SHOULDER.value gives the ID for the left shoulder landmark.
mp_pose.PoseLandmark.LEFT_ELBOW.value provides the ID for the left elbow landmark.

2. Landmark Object:
Each landmark object (like the ones in the landmarks list) typically contains:

x, y, z: The normalized coordinates of the landmark in the frame. "Normalized" means the values are between 0 and 1, with (0, 0) being the top-left corner and (1, 1) being the bottom-right corner of the frame.
visibility: A score between 0 and 1 representing the probability that the landmark is visible and not occluded. A higher score indicates higher confidence in visibility.

3. Working with Landmarks:
To access specific landmarks, you can index into the landmarks list with the appropriate landmark ID. For example:

To get the x-coordinate of the left shoulder: landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x
To get the visibility score of the left elbow: landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility

4. Error Handling with Landmarks:
In the code, there's a try and except block when extracting landmarks:

try:
    landmarks = results.pose_landmarks.landmark
    print(landmarks)
except:
    pass

This is important because sometimes Mediapipe might not detect any poses in a frame (maybe due to poor lighting, occlusion, or the person being out of the frame). In such cases, results.pose_landmarks would be None, and trying to access landmark on it would raise an error. The try and except block gracefully handles such scenarios by skipping the processing for that particular frame.

5. Visualization of Landmarks:
The mp_drawing.draw_landmarks() function is used to draw the detected landmarks and their connections on the image. The function is provided with:

The image to draw on.
The landmarks to draw.
The connections between landmarks (defined by mp_pose.POSE_CONNECTIONS).
Drawing specifications, such as color, line thickness, and circle radius.



In [ ]:
len(landmarks)

len(landmarks)

This line simply gets and displays the number of landmarks detected

In [ ]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

This block of code prints out all the pose landmarks that Mediapipe can detect. 
 


In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

mp_pose.PoseLandmark is an enumeration of all possible pose landmarks.

landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

This line gets the visibility score of the left shoulder landmark.
The visibility score indicates the probability of the landmark being visible in the image (not occluded).

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
This line retrieves details about the left elbow landmark, such as its coordinates and visibility score.

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

This line retrieves details about the left elbow landmark, such as its coordinates and visibility score.

# 3. Calculate Angles

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [ ]:
shoulder, elbow, wrist

In [ ]:
calculate_angle(shoulder, elbow, wrist)

In [ ]:
tuple(np.multiply(elbow, [640, 480]).astype(int))

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 4. Curl Counter

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()